<a href="https://colab.research.google.com/github/knh0503/CrimeManagementSystem/blob/main/offender_predict_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 범인 예측 시스템 개요

데이터베이스에는 범죄자들의 인적사항이 기록되어 있다.

범죄를 저지르고 도망친 용의자를 추적하기 위햐여 데이터베이스를 활용하여 인적사항과 위치기록을 대조하여 용의자의 위치를 예측하려고 한다.




# 데이터베이스 수정 및 추가


우선 데이터베이스에 offender_descriptioin 테이블에는 다음 정보들이 있어야 한다.

[id, 가해자 id, 나이, 성별, 키, 체중, 범죄 위치, 과거 체포 기록]

## region 테이블에 위치, 경도 추가

region은 0~77까지의 숫자로 이루어져 있다. 이에 위치, 경도를 추가하여 머신러닝 학습에 필요한 특성을 늘린다.

In [2]:
import psycopg2
import random

conn = psycopg2.connect(
    dbname="crimemanagementsystem",
    user="dbproject",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (::1), port 5432 failed: Cannot assign requested address
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
# 1. region 테이블에 latitude와 longitude 컬럼 추가
cursor.execute("""
    ALTER TABLE region
    ADD COLUMN IF NOT EXISTS latitude FLOAT,
    ADD COLUMN IF NOT EXISTS longitude FLOAT;
""")
conn.commit()

In [ ]:
cursor.execute("""
    update region
    set latitude = random() * (90 - (-90)) + (-90),
        longtitude = random() * (180 - (-180)) + (-180);
""")
conn.commit

## offender_description 테이블 추가

In [ ]:
# 1. offender_description 테이블 생성
cursor.execute("""
    CREATE TABLE offender_description (
        offender_id SERIAL PRIMARY KEY,
        age INT,
        gender bpchar(1),
        height FLOAT,
        weight FLOAT,
        latitude FLOAT,
        longitude FLOAT,
        previous_lat FLOAT[],
        previous_long FLOAT[]
    );
""")
conn.commit()

In [ ]:
# 2. 기존 offender 테이블 데이터 가져오기
cursor.execute("SELECT id, age, gender FROM offender")
offenders = cursor.fetchall()

In [ ]:
# 3. 각 범죄자 데이터 처리 및 offender_description에 삽입
for offender_id, age, gender in offenders:
    # 성별에 따른 키와 몸무게 랜덤 생성
    if gender == 'M':
        height = random.uniform(175, 195)  # 남성 키 범위
        weight = random.uniform(65, 95)   # 남성 몸무게 범위
    elif gender == 'F':
        height = random.uniform(150, 170)  # 여성 키 범위
        weight = random.uniform(40, 75)    # 여성 몸무게 범위
    else:
        height = None
        weight = None

    # offender_location에서 region 정보를 통해 latitude, longitude 설정
    cursor.execute(f"""
        SELECT r.latitude, r.longitude
        FROM offender_location ol
        JOIN region r ON ol.region_id = r.id
        WHERE ol.offender_id = {offender_id}
        LIMIT 1;
    """)
    location = cursor.fetchone()
    if location:
        latitude, longitude = location
    else:
        latitude = longitude = None

    # 과거 체포 위치 랜덤 생성 (1~5개)
    num_previous = random.randint(1, 5)
    previous_lat = [random.uniform(-90, 90) for _ in range(num_previous)]
    previous_long = [random.uniform(-180, 180) for _ in range(num_previous)]

    # offender_description 테이블에 데이터 삽입
    cursor.execute("""
        INSERT INTO offender_description (offender_id, age, gender, height, weight, latitude, longitude, previous_lat, previous_long)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (offender_id, age, gender, height, weight, latitude, longitude, previous_lat, previous_long))

# 변경 사항 저장 및 연결 종료
conn.commit()
cursor.close()
conn.close()

print("offender_description 테이블 생성 및 데이터 삽입 완료.")

offender_description 테이블 생성 및 데이터 삽입 완료.


## longtitude 오류

테이블을 확인해 보니 longtitude가 제대로 삽입되지 않았다.
    location = cursor.fetchone()
    if location:
        latitude, longitude = location
    else:
        latitude = longitude = None

이 부분에서 latitude, longtitude = location[0], location[1]이 되어야 하는데 longtitude 값이 무시된 것으로 보인다.
따라서 offender_description 테이블의 latitude와 region 테이블의 latitude가 같은 데이터에서 longtitude를 가져와 붙여준다.

In [ ]:
cursor.execute(f"""
UPDATE offender_description od
SET longitude = r.longitude
FROM region r
WHERE od.latitude = r.latitude;
""")

In [ ]:
conn.commit()
cursor.close()
conn.close()

여전히 제대로 경도가 입력되지 않았다. 알고보니 longtitude로 잘못 입력했다.

# 데이터베이스 테이블 csv 파일로 변경

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL 연결 설정
username = 'dbproject'  # PostgreSQL 사용자 이름
password = '1234'  # PostgreSQL 비밀번호
database_name = 'crimemanagementsystem'  # 데이터베이스 이름
host = 'localhost'  # 데이터베이스 호스트
port = '5432'  # PostgreSQL 기본 포트

# PostgreSQL 연결 URI 생성
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')

# SQL 쿼리 실행하여 데이터 가져오기
query = "SELECT * FROM offender_description"  # 원하는 쿼리 작성
df = pd.read_sql(query, engine)  # 쿼리 결과를 pandas DataFrame으로 읽어오기

# DataFrame을 CSV 파일로 저장
df.to_csv('offender_desciption_data.csv', index=False)  # index=False는 행 인덱스를 CSV에 포함하지 않도록 설정

# 머신러닝 모델 학습 및 훈련

HistGradientBoosting 머신러닝을 학습시키는 건 **구글 코랩**에서 진행한다.

## CSV 파일을 읽어 데이터프레임으로 변환

In [2]:
import pandas as pd
import numpy as np

# offender_description csv 파일 읽기
offender_data = pd.read_csv('/content/offender_desciption_data.csv')

### 이전 위치 (위도, 경도)를 평균으로 처리하기

이전 위치 기록의 경우, 범인마다 1~5개의 이전 위치 기록이 배열로 저장되어 있다. 이번에는 이 기록들을 평균으로 만들어 사용한다.

***이후에 RNN 등 시퀀스 모델로 확장하여 개발해보록 하자***

In [3]:
offender_data['prev_lat_mean'] = offender_data['previous_lat'].apply(lambda x: np.mean(eval(x)))

In [4]:
offender_data['prev_long_mean'] = offender_data['previous_long'].apply(lambda x: np.mean(eval(x)))

1. offender_data['previous_lat']
  - previous_lat 컬럼에는 각 범인의 이전 위도 기록이 문자열 형태로 저장되어 있다.

In [5]:
offender_data['previous_lat'].iloc[0]

'[-89.26464093736418, -88.74733932514289, -5.802098639159098, -72.42534893338004]'

2. eval(x)

-  eval 함수는 문자열로 저장된 리스트를 실제 리스트로 변환한다.



In [5]:
eval('[-89.26464093736418, -88.74733932514289, -5.802098639159098, -72.42534893338004]')

[-89.26464093736418,
 -88.74733932514289,
 -5.802098639159098,
 -72.42534893338004]

3. np.mean()

- 리스트의 평균값을 계산

4. apply(lambda x: ...)

- apply() 는 previous_lat 칼럼의 각 행에 대해 변환을 적용
- 각 범인의 이전 위도 리스트의 평균값을 계산하여 "prev_lat_mean" 이라는 개로운 칼럼에 저장

In [6]:
print(offender_data.head())

   offender_id  age gender      height     weight   latitude  longtitude  \
0      8996917   57      F  160.817843  71.691094  30.385876  127.738074   
1      9925096   59      M  175.440508  67.921761 -46.641767  -80.068478   
2      8992514   23      F  165.804355  45.985506  30.385876  127.738074   
3      9387503   90      F  161.242845  59.904000  57.978040  134.008790   
4      9119534   75      F  161.178803  42.262891 -39.424884  -68.071145   

                                        previous_lat  \
0  [-89.26464093736418, -88.74733932514289, -5.80...   
1  [10.71846859423664, 36.160539081581234, 69.081...   
2                                [50.62854706596346]   
3  [18.98301898918487, 49.90367591267071, 37.9937...   
4  [-26.13505772287092, 85.2627941343581, -83.457...   

                                       previous_long  prev_lat_mean  \
0  [23.316054997158204, -7.41891190039496, -98.77...     -64.059857   
1  [35.85286204718818, 114.60493484712606, 169.84...      11.081

### gender 인코딩

In [8]:
offender_data['gender_encoded'] = offender_data['gender'].map({'M':0, 'F':1})

## 특성, 타겟 정의 및 데이터 분할

In [30]:
X = offender_data[['age', 'gender_encoded', 'height', 'weight', 'prev_lat_mean', 'prev_long_mean']]
y = offender_data[['latitude', 'longtitude']]

In [31]:
print(X.head())
print(y.head())

   age  gender_encoded      height     weight  prev_lat_mean  prev_long_mean
0   57               1  160.817843  71.691094     -64.059857        8.752633
1   59               0  175.440508  67.921761      11.081764       86.171804
2   23               1  165.804355  45.985506      50.628547      164.471244
3   90               1  161.242845  59.904000      31.510347        1.478239
4   75               1  161.178803  42.262891       9.118236      -46.528076
    latitude  longtitude
0  30.385876  127.738074
1 -46.641767  -80.068478
2  30.385876  127.738074
3  57.978040  134.008790
4 -39.424884  -68.071145


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
print(X_train.describe())
print(X_test.describe())
print(y_train.describe())
print(y_test.describe())

                 age  gender_encoded         height         weight  \
count  302068.000000   302068.000000  302068.000000  302068.000000   
mean       55.061542        0.500712     172.481492      68.744246   
std        23.432720        0.500000      13.764376      14.694909   
min        11.000000        0.000000     150.000031      40.000130   
25%        35.000000        0.000000     159.988209      57.428688   
50%        55.000000        1.000000     169.973300      69.638551   
75%        75.000000        1.000000     184.986178      80.032137   
max        97.000000        1.000000     194.999705      94.999779   

       prev_lat_mean  prev_long_mean  
count  302068.000000   302068.000000  
mean        0.017662        0.110525  
std        35.136209       70.241871  
min       -89.999632     -179.996324  
25%       -22.875606      -45.760248  
50%        -0.031800        0.379127  
75%        22.858801       45.825871  
max        89.998775      179.997054  
                ag

## HistGradientBoostingRegressor 모델 정의 및 훈련

히스토그램그래디언트부스팅 알고리즘은 다중 출력을 허가하지 않는다. 따라서 MultiOutputRegressor로 다중출력을 해야한다.


In [44]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# 다중 출력 분류기 생성
location_model = MultiOutputRegressor(HistGradientBoostingRegressor())

# 모델 훈련
location_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=HistGradientBoostingRegressor())

# 모델 평가

In [45]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model():
    # 학습 데이터에 대한 예측
    y_train_pred = location_model.predict(X_train)

    # 테스트 데이터에 대한 예측
    y_test_pred = location_model.predict(X_test)

    # R2 점수 계산
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # RMSE 계산 (mean_squared_error)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # 교차 검증 점수
    cv_scores = cross_val_score(location_model, X, y, cv=5)

    print("모델 성능 평가:")
    print(f"Train R² Score: {train_r2:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"5-Fold CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [46]:
evaluate_model()

모델 성능 평가:
Train R² Score: 0.0014
Test R² Score: -0.0002
Train RMSE: 77.7075
Test RMSE: 77.7637
5-Fold CV Score: -0.0003 (+/- 0.0003)


# 문제 분석

R2 점수가 0에 가깝다. 이는 모델이 데이터를 제대로 학습하지 못하고 있거나 예측이 거의 평균값에 가깝다는 것이다.

## 데이터 스케일링

In [53]:
from sklearn.preprocessing import StandardScaler

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 훈련
# 다중 출력 분류기 생성
location_model = MultiOutputRegressor(HistGradientBoostingRegressor(
    max_iter=500,                        # 부스팅 스테이지 수(에포크 횟수)
    max_depth=5,                        # 트리 최대 깊이
    min_samples_leaf=5,                       # 리프 노드의 최소 샘플 수
    learning_rate=0.05,                   # 학습률
    l2_regularization=1.0,               # L2 정규화 강도
    random_state=42
))

# 모델 훈련
location_model.fit(X_train_scaled, y_train)

MultiOutputRegressor(estimator=HistGradientBoostingRegressor(l2_regularization=1.0,
                                                             learning_rate=0.05,
                                                             max_depth=5,
                                                             max_iter=500,
                                                             min_samples_leaf=5,
                                                             random_state=42))

In [54]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model():
    # 학습 데이터에 대한 예측
    y_train_pred = location_model.predict(X_train_scaled)

    # 테스트 데이터에 대한 예측
    y_test_pred = location_model.predict(X_test_scaled)

    # R2 점수 계산
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # RMSE 계산 (mean_squared_error)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # 교차 검증 점수
    cv_scores = cross_val_score(location_model, X_train_scaled, y_train, cv=5)

    print("모델 성능 평가:")
    print(f"Train R² Score: {train_r2:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"5-Fold CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [55]:
evaluate_model()

모델 성능 평가:
Train R² Score: 0.0005
Test R² Score: -0.0000
Train RMSE: 77.7435
Test RMSE: 77.7578
5-Fold CV Score: -0.0001 (+/- 0.0002)


## 랜덤하게 생성된 데이터

이전에 데이터 베이스를 만들 때, 키, 체중, 위도, 경도, 이전 위치 정보를 파이썬 random 함수를 사용하여 핸덤하게 생성하였다.

랜덤 데이터는 규칙성이나 패턴이 거의 없다. 머신러닝의 경우 데이터를 학습하여 입력과 출력 사이의 패턴을 찾는데 랜덤 데이터는 의미 있는 관계를 찾을 수 없기 때문에 모델이 학습할 수 있는 정보가 거의 없다.

따라서 모델은 평균값을 뱉어낼 수 밖에 없고 r2점수도 0에 가까운 것이다.

### 패턴이 있는 데이터 생성

In [64]:
import numpy as np
import pandas as pd

# 직선 이동 패턴 데이터 생성 함수
def generate_linear_movement_data(num_samples):
    np.random.seed(42)  # 결과 재현을 위한 시드 고정

    # 입력 데이터 (특성): 나이, 성별, 키, 몸무게, 이전 위치 (latitude, longitude)
    age = np.random.randint(20, 80, size=num_samples)                   # 나이 (20~80세)
    gender = np.random.randint(0, 2, size=num_samples)                  # 성별 (0=남성, 1=여성)
    height = np.random.uniform(150, 190, size=num_samples)              # 키 (150~190cm)
    weight = np.random.uniform(50, 100, size=num_samples)               # 몸무게 (50~100kg)

    # 초기 위치 (랜덤 위도와 경도)
    initial_lat = np.random.uniform(-50, 50, size=num_samples)          # 초기 위도
    initial_long = np.random.uniform(-100, 100, size=num_samples)       # 초기 경도

    # 이동 방향과 속도 (각 샘플마다 랜덤한 방향과 이동량 설정)
    delta_lat = np.random.uniform(-0.5, 0.5, size=num_samples)          # 위도 변화량
    delta_long = np.random.uniform(-0.5, 0.5, size=num_samples)         # 경도 변화량

    # 이전 위치 (이전 위치 = 초기 위치 + 변위)
    prev_lat = initial_lat - delta_lat                                  # 이전 위도
    prev_long = initial_long - delta_long                               # 이전 경도

    # 현재 위치 (목표값, 즉 예측해야 하는 값)
    current_lat = initial_lat + delta_lat                               # 현재 위도 (직선 이동)
    current_long = initial_long + delta_long                            # 현재 경도 (직선 이동)

    # 입력 데이터와 출력 데이터 결합
    X = pd.DataFrame({
        'age': age,
        'gender_encoded': gender,
        'height': height,
        'weight': weight,
        'prev_lat': prev_lat,
        'prev_long': prev_long
    })

    y = pd.DataFrame({
        'latitude': current_lat,
        'longitude': current_long
    })

    return X, y

# 데이터 생성
X, y = generate_linear_movement_data(10000)

# 데이터 확인
print(X.head())
print(y.head())


   age  gender_encoded      height     weight   prev_lat  prev_long
0   58               0  166.681949  51.686505  20.860785 -41.657359
1   71               0  164.018808  77.558230  -4.015576   5.236594
2   48               0  184.352557  98.476328   5.168446 -89.146978
3   34               0  155.061778  99.062230  -4.877432  98.426617
4   62               0  172.625795  64.087629  38.953544 -15.666560
    latitude  longitude
0  20.845721 -41.018386
1  -4.181113   5.879968
2   6.128016 -89.474994
3  -4.042436  98.328208
4  39.117956 -14.713385


In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# 다중 출력 분류기 생성
location_model = MultiOutputRegressor(HistGradientBoostingRegressor())

# 모델 훈련
location_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=HistGradientBoostingRegressor())

In [67]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

def evaluate_model():
    # 학습 데이터에 대한 예측
    y_train_pred = location_model.predict(X_train)

    # 테스트 데이터에 대한 예측
    y_test_pred = location_model.predict(X_test)

    # R2 점수 계산
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # RMSE 계산 (mean_squared_error)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # 교차 검증 점수
    cv_scores = cross_val_score(location_model, X, y, cv=5)

    print("모델 성능 평가:")
    print(f"Train R² Score: {train_r2:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"5-Fold CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [68]:
evaluate_model()

모델 성능 평가:
Train R² Score: 0.9998
Test R² Score: 0.9997
Train RMSE: 0.5422
Test RMSE: 0.6202
5-Fold CV Score: 0.9997 (+/- 0.0000)


완벽하다.

# 예측 함수 정의

In [79]:
def predict_offender_location(age, gender, height, weight, prev_lat, prev_long):

    gender_mapping = {"M":0, "F":1}
    gender_encoded = gender_mapping[gender]

    # 예측 수행
    prediction = location_model.predict([[age, gender_encoded, height, weight, prev_lat, prev_long]])[0]
    return np.round(prediction, 2)

In [81]:
# 예측 예시
print("\n예측 예시:")
example_cases = [
    (20, "M", 180, 65, 55, 20),
    (65, "F", 160, 45, 23, -50),
    (70, "M", 170, 88, -45, -66)
]

for case in example_cases:
    age, gender, height, weight, prev_lat, prev_long = case
    predicted = predict_offender_location(age, gender, height, weight, prev_lat, prev_long)
    print(f"age: {age}, gender: {gender}, height: {height}, weight: {weight}, prev_lat: {prev_lat}, prev_long: {prev_long}")
    print(f"예측된 위치: {predicted}\n")


예측 예시:
age: 20, gender: M, height: 180, weight: 65, prev_lat: 55, prev_long: 20
예측된 위치: [49.36 20.12]

age: 65, gender: F, height: 160, weight: 45, prev_lat: 23, prev_long: -50
예측된 위치: [ 23.   -49.52]

age: 70, gender: M, height: 170, weight: 88, prev_lat: -45, prev_long: -66
예측된 위치: [-45.22 -66.47]



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

# 모델 저장

In [83]:
import joblib

# 최적 모델을 저장
joblib.dump(location_model, 'offender_location_predict.pkl')

['offender_location_predict.pkl']

In [84]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/crime_predictor_model_v2.ipynb"

[NbConvertApp] WARNING | pattern '/content/drive/MyDrive/Colab Notebooks/crime_predictor_model_v2.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equival